In [1]:

import os
os.chdir(os.path.dirname(os.getcwd()))

In [2]:
os.getcwd()

'/Users/chrisculley/Google Drive/PhD_Projects/Weight_Fix_Networks'

In [3]:
import torch 
from Models.Pretrained_Model_Template import Pretrained_Model_Template 
from inspect import getmembers, isfunction
import numpy as np
from scipy.stats import entropy
import os
import torchvision.models as models
from Datasets import cifar10
import pytorch_lightning as pl
import pandas as pd
import copy
from collections import OrderedDict
import zipfile
import matplotlib.pyplot as plt
from Comparison_Models.APoT_Quantization.ImageNet.models import resnet as apot_res
from Comparison_Models.APoT_Quantization.ImageNet.models import quant_layer
import torchvision.models as models
from dahuffman import HuffmanCodec
import copy
from Models.Pretrained_Model_Template import Pretrained_Model_Template
from Datasets import cifar10, mnist, imagenet
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
import os
from torch.utils.tensorboard import SummaryWriter
from pytorch_lightning.callbacks import ModelCheckpoint
import argparse
import torchvision.models as models
from Models.All_Conv_4 import All_Conv_4
import re
import numpy as np

In [4]:
data = imagenet.ImageNet_Module()
data.setup()

def test_the_model(model, name):
    trainer = pl.Trainer(gpus=-1, max_epochs =0, gradient_clip_val=0.5, num_sanity_val_steps = 0, checkpoint_callback=False)   
    trainer.fit(model, data)
    model.eval()
    acc = trainer.test(ckpt_path=None)[0]
    return (acc['test_acc_epoch'], acc['test_top5_acc_epoch'])
    
def grab_wfn(file, version):
    if version == 18:
        model = models.resnet18(pretrained=False)
        model_name = 'resnet18'
        model.name = model_name
    model = Pretrained_Model_Template(copy.deepcopy(model), 0, data, 0.0002, False, 'ADAM')
    model.load_from_checkpoint(checkpoint_path=f'{os.getcwd()}/WFN_Model_Saves/ImageNet/{file}', max_epochs=model.max_epochs, original_model=model.pretrained, data_module=model.data_module,scheduler=None, lr= model.lr, opt= model.opt)
    model.set_up('relative', 'pow_2_add', 0.015, 0, 0, 0, 0, True)
    model.reset_optim(1)
    model.print_unique_params()
    return model
model_list = {} 

In [5]:
def flatten_the_network(s_dict, inc_linear = False, inc_bn = False):
    weights = np.zeros(0)
    layer_weights = {} 
    for k in s_dict:
        if 'alpha' not in k and 'conv'in k or (inc_linear and ('fc' in k or 'linear' in k)) or ('bn' in k and inc_bn) and 'running' not in k and 'tracked' not in k:
            try:
                c = s_dict[k].detach().cpu().numpy().flatten()
            except:
                c = s_dict[k].flatten()
            weights = np.append(weights, c)
            layer_weights[k] = c
    return weights, layer_weights
            

def entropy_of_params(model, inc_linear = False, inc_bn = False):
    flat, _ = flatten_the_network(model, inc_linear, inc_bn)
    v,c =  np.unique(flat, return_counts = True)
    c = np.array(c) / np.sum(c)
    return entropy(c, base=2), len(c), len(flat)

def entropy_of_params_non_zero(model, inc_linear = False, inc_bn = False):
    flat, _ = flatten_the_network(model, inc_linear, inc_bn)
    v,c =  np.unique(flat, return_counts = True)
    try:
        c = np.delete(c, np.where(v == 0.0)[0][0])
    except:
        print('no zeros')
    c = np.array(c) / np.sum(c)
    return entropy(c, base=2), len(c)

def entropy_per_layer(model, inc_linear = False, inc_bn = False):
    _, layers = flatten_the_network(model, inc_linear, inc_bn )
    
    for k in layers:
        v,c =  np.unique(layers[k], return_counts = True)
        c = np.array(c) / np.sum(c)
        layers[k] = (entropy(c, base=2), len(c))
    return layers 
        
        
def num_of_params_in_network(model, inc_linear):
    flat, _ = flatten_the_network(model, inc_linear)
    return len(np.unique(flat))

In [6]:
resnet_version = 18 

model = grab_wfn('res18_0.01', resnet_version)


/opt/anaconda3/lib/python3.7/site-packages/deprecate/deprecation.py:115: LightningDeprecationWarning: The `Accuracy` was deprecated since v1.3.0 in favor of `torchmetrics.classification.accuracy.Accuracy`. It will be removed in v1.5.0.
  stream(template_mgs % msg_args)


BATCH NORM included
138
21
34
122
29
48
86
27
42
85
19
34
66
29
41
72
15
33
82
28
42
93
34
42
76
18
38
53
43
56
71
19
50
63
28
45
52
29
45
54
26
61
53
41
69
48
28
60
53
31
49
50
30
49
57
23
59
43
11
33
96
13


In [17]:
np.save('flattened_model', flatten_the_network(model.state_dict())[0])

In [7]:
res = pd.DataFrame()
for i in os.listdir('WFN_Model_Saves/ImageNet/'):
    folder = i
    if f'res{resnet_version}' in folder:
        print(i)
        try:
            model = grab_wfn(folder, version = resnet_version)
            model_list[i]= model
            e, n  = entropy_of_params_non_zero(model.state_dict(), inc_bn=True, inc_linear=True)
          #  top1, top5 = test_the_model(model, folder)
            res = res.append({'name': folder, 'H': e, 'Np' : n}, ignore_index=True)
            print(res)
        except Exception as e:
            print('broke', e)
        

res18_0.015
BATCH NORM included
74
11
21
68
14
29
44
12
26
41
10
28
32
15
24
37
7
23
40
11
22
48
13
22
38
8
32
24
15
41
34
9
33
32
11
26
23
10
26
22
12
43
21
14
46
18
12
41
24
11
34
21
10
34
27
11
40
17
8
11
41
10
          H    Np         name
0  2.726613  90.0  res18_0.015
res18_entropy.csv
broke invalid load key, '\xef'.
res18_0.01
BATCH NORM included
138
21
34
122
29
48
86
27
42
85
19
34
66
29
41
72
15
33
82
28
42
93
34
42
76
18
38
53
43
56
71
19
50
63
28
45
52
29
45
54
26
61
53
41
69
48
28
60
53
31
49
50
30
49
57
23
59
43
11
33
96
13
          H     Np         name
0  2.726613   90.0  res18_0.015
1  3.021636  164.0   res18_0.01
res18_0.0075
BATCH NORM included
154
22
29
130
30
39
78
28
37
81
22
33
57
31
28
67
17
34
73
30
35
93
21
35
77
19
45
42
36
58
66
20
50
53
30
42
39
11
42
42
27
75
40
36
79
35
30
76
40
35
65
35
16
65
47
30
74
29
13
42
97
19
          H     Np          name
0  2.726613   90.0   res18_0.015
1  3.021636  164.0    res18_0.01
2  4.150800  193.0  res18_0.0075
res18_

In [8]:
def grab_apot(bit):
    if resnet_version == 18:
        model_apot = apot_res.resnet18(pretrained=False, bit = bit)
        
    else:
        model_apot = apot_res.resnet50(pretrained=False, bit = bit)
    sd = torch.load(f'Comparison_Models/APoT_Quantization/res{resnet_version}_{bit}bit_best.pth.tar', map_location=torch.device('cpu'))['model']

    new_dict = {} 
    for t in sd.keys():
        s = t.replace("module.", "")
        new_dict[s] = sd[t]
        sd[t] = None

    model_apot.load_state_dict(new_dict)
    model_apot.show_params()
    weight_dict = {} 
    for n, p in model_apot.named_modules():
          try:
            for a,b in p.named_parameters():
                if a == 'weight':
                    b.data = p.get_the_weight_values()
          except:
            continue

    return model_apot


def create_full_wfn_version(model):
    model = Pretrained_Model_Template(copy.deepcopy(model), 1, data, 3e-4, False, 'ADAM')
    model.set_up('relative', 'pow_2_add', 0, 0, 0, 0, 0, 0)
    model.reset_optim(0)
    return model 

def move_to_wfn(model_dict):
    if resnet_version == 18:
        model = models.resnet18(pretrained=False)
    else:
        model = models.resnet50(pretrained=False)
    standard_model_keys = model.state_dict().keys()
    new_state_dict = {} 
    for key in model_dict:
        if key in standard_model_keys:
            new_state_dict[key] = model_dict[key]

    model_name = f'resnet{resnet_version}'
    model.name = model_name
    model.load_state_dict(new_state_dict)
    model = Pretrained_Model_Template(copy.deepcopy(model), 1, data, 3e-4, False, 'ADAM')
    
    return model

for x in range(3, 6):
    ap = grab_apot(x)
    name = 'APoT '+str(x) + 'bit'
    ap.name = name
 #   results[name] = test_the_model(create_full_wfn_version(ap), name)
    model_list[name] = move_to_wfn(grab_apot(x).state_dict()) 
    
if resnet_version == 18:
    model_orig = apot_res.resnet18(pretrained=True, bit = 32)
else:
    model_orig = apot_res.resnet50(pretrained=True, bit = 32)

model_list['baseline'] = model_orig

GETTING THE FC LAYER WEIGHTS
GETTING THE FC LAYER WEIGHTS
GETTING THE FC LAYER WEIGHTS
GETTING THE FC LAYER WEIGHTS
GETTING THE FC LAYER WEIGHTS
GETTING THE FC LAYER WEIGHTS


In [10]:
model_list.keys()

dict_keys(['res18_0.015', 'res18_0.01', 'res18_0.0075', 'APoT 3bit', 'APoT 4bit', 'APoT 5bit', 'baseline'])

In [17]:
print(model_list['res18_0.015'].pretrained)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [15]:
print(model_list['baseline'])

ResNet(
  (conv1): first_conv(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): QuantConv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): QuantConv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): QuantConv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): Re

# CKA

In [ ]:
# attach a hook to the activations 
# cka to the activations 


from torch_cka import CKA
model1 = model_list['res18_0.015'] # Or any neural network of your choice
model2 = model_list['baseline']

dataloader = data.val_dataloader()


cka = CKA(model1, model2,
          model1_name="WFN_0.015",   # good idea to provide names to avoid confusion
          model2_name="Baseline",   
  #        model1_layers=layer_names_resnet18, # List of layers to extract features from
  #        model2_layers=layer_names_resnet34, # extracts all layer features by default
          device='cpu')

cka.compare(dataloader) # secondary dataloader is optional

results = cka.export()  # returns a dict that contains model names, layer names
                        # and the CKA matrix

| Comparing features |:   3%|▎         | 170/5000 [21:58<2:51:21,  2.13s/it]  

In [32]:
print(model_list['res18_0.015'].pretrained)
print(model_list['baseline'])

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [7]:

model_orig = apot_res.resnet50(pretrained=True, bit = 32)
model_orig.name = 'baseline'
model_list['baseline'] = model_orig
#results['baseline'] = test_the_model(create_full_wfn_version(model_list['baseline']), 'baseline')

In [20]:

model_entropy = {} 
model_entropy_layer = {} 
model_total_unique = {} 

for x in model_list:
    model_entropy_layer[x] = entropy_per_layer(model_list[x].state_dict(), inc_bn=True, inc_linear=True)
    model_entropy[x], model_total_unique[x] = entropy_of_params_non_zero(model_list[x].state_dict(), inc_bn=True, inc_linear=True)

no zeros


In [51]:
print(model_total_unique, model_entropy)

{'res50_0.01': 199, 'res50_0.015': 125, 'res50_0.0075': 261} {'res50_0.01': 4.00014828710685, 'res50_0.015': 3.553006780058877, 'res50_0.0075': 4.119458326257029}


In [ ]:
f, ax = plt.subplots(3, figsize=(15,15))

details_for_plot = {}
for i, x in enumerate(model_list):
    if x == 'baseline':
        continue
    sizes = [] 
    entropies = [] 
    names = [] 
    for t in model_entropy_layer[x]:
        names.append(t.replace('pretrained.', ""))
        sizes.append(model_entropy_layer[x][t][1])
        entropies.append(model_entropy_layer[x][t][0])
  #  details_for_plot[x] = (names, entropies, names)
   
    ax[2].set_xticks(range(0, len(entropies)))
    ax[1].set_xticks(range(0, len(entropies)))
    ax[0].set_xticks(range(0, len(entropies)))
    if 'WFN' in x:
        c = 'red'
    else:
        c = 'blue'
    ax[0].plot(sizes, linewidth=4, label=x)
    ax[1].plot(np.cumsum(sizes), linewidth=4, label=x)
    ax[2].plot(entropies, linewidth=4, label=x)
    if i == 0:
        ax[2].set_xticklabels(names, rotation='vertical')
    else:
        ax[0].set_xticklabels("")
        ax[1].set_xticklabels("")
    ax[0].set_ylabel('Layer Unique Param Count',fontsize=15)
    ax[1].set_ylabel('Cumulative Unique Param Count',fontsize=15)
    ax[2].set_ylabel('Layer Entropy',fontsize=15)
    ax[0].legend()
        
plt.tight_layout()
plt.savefig('layer-wise-param-count.pdf')

    


In [9]:
def flatten_the_network(model):
    params = []
    for n, p in model.named_modules():
      if isinstance(p, (torch.nn.Conv2d, torch.nn.Linear, torch.nn.BatchNorm2d)):
        for nn, pp in p.named_parameters():
            params.extend(pp.detach().numpy().flatten())
    return torch.Tensor(params)
            
    
def get_the_weight_dist_percentage(model):
    f = flatten_the_network(model)
    values = np.unique(f, return_counts = True)
    dist = {}
    tot = np.sum(values[1])
    for i,t in enumerate(values[0]):
        dist[t] = np.round(100*(values[1][i] / tot), 1)
    values = values[0]
    return values, dist
 

class Activation_Hook():
    def __init__(self,name,  module):
        self.hook = module.register_forward_hook(self.hook_function)
        self.activations = []
        self.size = 0
        self.name = name
        
    def hook_function(self, module, input, output):
        self.activations.append(output)
        self.size = output.size()
    def close(self):
        self.hook.remove()


def add_hook_to_model(model, hook):
    hooks = {}
    for n,p in model.named_modules():
        if isinstance(p, (torch.nn.Conv2d, torch.nn.Linear, torch.nn.BatchNorm2d)):
            hooks[n] = hook(n,p)
            
    return hooks
        
    
def create_hooks(model):
    hook_set =  add_hook_to_model(model, Activation_Hook)
    x = torch.rand((1,3, 256, 256))
    model(x)
    return hook_set#, hook_set_orig

def weight_use_number(model, v, hook_set):
    activation_use_dict = {str(x):0 for x in v}
    for n, p in model.named_modules():
      if isinstance(p, (torch.nn.Conv2d, torch.nn.Linear, torch.nn.BatchNorm2d)):
        for nn, pp in p.named_parameters():
            if isinstance(p, torch.nn.Conv2d):
                s = hook_set[n].size
                batch_num = s[0]
                sp = s[-2] * s[-1] # get the size of output image 
                if 'weight' in nn:
                                val,count = np.unique(pp.cpu().detach().flatten(), return_counts = True)
                                for i,v in enumerate(val):
                                    activation_use_dict[str(v)] += count[i]*sp*batch_num
                elif 'bias' in nn:
                                val,count = np.unique(pp.cpu().detach().flatten(), return_counts = True)
                                for i,v in enumerate(val):
                                    activation_use_dict[str(v)] += count[i]*batch_num
            elif isinstance(p, torch.nn.Linear):
                                val,count = np.unique(pp.cpu().detach().flatten(), return_counts = True)
                                for i,v in enumerate(val):
                                    activation_use_dict[str(v)] += count[i]
            else:
                val,count = np.unique(pp.cpu().detach().flatten(), return_counts = True)
                for v in val:
                    activation_use_dict[str(v)] = 1
    return activation_use_dict

def get_huffman_encoding(activation_use_dict, v):
    codec = HuffmanCodec.from_frequencies(activation_use_dict)
    cost = {symbol:bits for symbol, (bits, val) in codec._table.items()}
    return cost

def get_combined_cost(activation_use_dict, huffman_bit_cost):
    total_cost = {} 
    for weight in huffman_bit_cost:
        if weight in activation_use_dict:
            total_cost[weight] = int(activation_use_dict[weight]) * int(huffman_bit_cost[weight])
    return total_cost
        
     
def get_the_representation_cost(model):
    model = model.cpu()
    v, _ = get_the_weight_dist_percentage(model)
    hook_set = create_hooks(model)
    activation_use_dict = weight_use_number(model, v, hook_set)
    huffman_encoding_cost = get_huffman_encoding(activation_use_dict, v)
    total_costs = get_combined_cost(activation_use_dict, huffman_encoding_cost)
    return sum(total_costs.values())



In [53]:
print(model_list.keys())

dict_keys(['res50_0.01', 'res50_0.015', 'res50_0.0075'])


In [10]:
representation_cost = {} 

for k in model_list:
    print(k)
    representation_cost[k]= get_the_representation_cost(model_list[k])
    
print(representation_cost)

res50_0.01
res50_0.015
res50_0.0075
baseline
{'res50_0.01': 339010270, 'res50_0.015': 250481463, 'res50_0.0075': 366592193, 'baseline': 123561335650}


In [ ]:
for k in model_list:
    torch.save(model_list[k].state_dict(), 'Compressed_Versions/'+k)
    

In [ ]:
for k in representation_cost:
    print(k, representation_cost[k]/representation_cost['baseline'])

In [11]:
for t in representation_cost:
    print(t, representation_cost[t] / representation_cost['baseline'])

res50_0.01 0.002743659804392864
res50_0.015 0.002027183193531625
res50_0.0075 0.00296688434995887
baseline 1.0
